In [6]:
!pip install diffusers transformers torch gradio accelerate pillow -q



In [8]:
import torch
import gradio as gr
from diffusers import StableDiffusionPipeline

device = "cuda" if torch.cuda.is_available() else "cpu"

pipe = StableDiffusionPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    torch_dtype=torch.float16
).to(device)

STYLE_PRESETS = {
    "Realistic": "ultra realistic, high detail, sharp focus, product photography",
    "Anime": "anime style, vibrant colors, studio ghibli, illustration",
    "Cyberpunk": "cyberpunk, neon lights, futuristic city, sci-fi",
    "Oil Painting": "oil painting, textured brush strokes, classical art"
}


def generate(prompt, style, seed, steps, guidance):
    styled_prompt = f"{prompt}, {STYLE_PRESETS[style]}"

    generator = torch.Generator(device).manual_seed(seed)

    image = pipe(
        styled_prompt,
        num_inference_steps=steps,
        guidance_scale=guidance,
        generator=generator
    ).images[0]

    return image



demo = gr.Interface(
    fn=generate,
    inputs=[
        gr.Textbox(label="Prompt", placeholder="e.g. iPhone product shot"),
        gr.Dropdown(
            choices=list(STYLE_PRESETS.keys()),
            value="Realistic",
            label="Style"
        ),
        gr.Slider(0, 9999, value=1024, step=1, label="Seed"),
        gr.Slider(10, 50, value=25, step=1, label="Steps"),
        gr.Slider(5, 15, value=7.5, step=0.5, label="Guidance Scale")
    ],
    outputs=gr.Image(label="Generated Image"),
    title="Prompt-Conditioned Stable Diffusion Generator",
    description="Style presets control aesthetics, while steps and guidance control quality and prompt adherence."
)



demo.launch(share=True)


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://5aef5856b9a84dd57f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
